In [14]:
import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv('.env')
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [15]:
def get_input_variables():
    variables = []
    while True:
        variable_name = input("Enter variable name (or type 'done' to finish): ")
        if variable_name.lower() == 'done':
            break
        variable_value = input("Enter variable value: ")
        variables.append((variable_name, variable_value))
    
    json_data = {}
    for var_name, var_value in variables:
        json_data[var_name] = var_value
        
    return json_data

def get_mongodb_schema_from_user():
    schema = {}
    while True:
        collection_name = input("Enter collection name (or type 'done' to finish): ")
        if collection_name.lower() == 'done':
            break
        fields = []
        while True:
            field_name = input(f"Enter field name for collection '{collection_name}' (or type 'done' to finish): ")
            if field_name.lower() == 'done':
                break
            field_type = input(f"Enter field type for field '{field_name}': ")
            fields.append({field_name: field_type})
        schema[collection_name] = fields
    return schema


In [16]:
# task = input("What do you want to do?")
# function_parameters = get_input_variables()
# mongodb_schema = get_mongodb_schema_from_user()

In [17]:
task = "Give a list of users that have bought the given product"
function_parameters = {"product_id": "apple"}
mongodb_schema = {
  "users": [
    { "user_id": "string", "username": "string", "email": "string", "password": "string", "created_at": "datetime" }
  ],
  "products": [
    { "product_id": "string", "name": "string", "description": "string", "price": "double", "stock_quantity": "int", "created_at": "datetime" }
  ],
  "orders": [
    { "order_id": "string", "user_id": "string", "products": [{"product_id": "string", "quantity": "int"}], "total_amount": "double", "order_date": "datetime" }
  ]
}

In [18]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")
prompt = f'''
                    
Task Description: Database Schema to Function Code Generator

Generate code for a function in python that performs the task specified on the given database schema. The function should be able to perform the specified task on the given database schema using the provided parameters.  The function should return something relevant to the task.

Input function parameters: {function_parameters}
Input database schema: {mongodb_schema}
Input task: {task}



'''
result = llm.invoke(prompt)
print(result.content)

```python
def get_users_who_bought_product(product_id):
    """
    Given a product id, return a list of users that have bought the given product

    Args:
        product_id (str): The id of the product

    Returns:
        list: A list of users that have bought the given product
    """

    # Get all the orders for the given product
    orders = db.orders.find({"products.product_id": product_id})

    # Get all the users who have placed the orders
    users = []
    for order in orders:
        users.append(db.users.find_one({"user_id": order["user_id"]}))

    return users
```
